IMDB Baseline Model (Structured Data Only)

This notebook trains a baseline model to predict whether a movie is highly rated or not (`label` column), using only structured features from the dataset.

We are **NOT** using review text or sentiment analysis here. This version focuses on numerical and categorical data like votes, year, and Rotten Tomatoes ratings.


## **Step 1: Load Data & Initialize Spark**

We start by launching a Spark session and loading the cleaned dataset. The dataset includes movie-level information like `numVotes`, `genre`, `tomatometer_rating`, etc.


In [11]:
from pyspark.sql import SparkSession
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Start Spark session
spark = SparkSession.builder.appName("IMDB_Baseline_Model").getOrCreate()

# Load final cleaned dataset (structured data only)
df = spark.read.csv("final_cleaned_df.csv", header=True, inferSchema=True)

# Quick check on columns
df.printSchema()
df.show(5)


root
 |-- tconst: string (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- numVotes: integer (nullable = true)
 |-- label: boolean (nullable = true)
 |-- genre: string (nullable = true)
 |-- content_rating: string (nullable = true)
 |-- production_company: string (nullable = true)
 |-- tomatometer_status: integer (nullable = true)
 |-- tomatometer_rating: integer (nullable = true)
 |-- audience_status: integer (nullable = true)
 |-- audience_rating: integer (nullable = true)
 |-- review_score: string (nullable = true)
 |-- like_count: double (nullable = true)
 |-- label_int: integer (nullable = true)
 |-- reviews: string (nullable = true)
 |-- review_lemmatized: string (nullable = true)

+---------+------------------+----+--------+-----+-------+--------------+------------------+------------------+------------------+---------------+---------------+------------+----------+---------+--------------------+--------------------+
|   tconst|


## **Step 2: Encode Categorical Variables**

Some columns are categorical (e.g., `genre`, `tomatometer_status`, `audience_status`).  
These are converted into numerical values using **StringIndexer** so they can be used as inputs to the machine learning model.



In [4]:
from pyspark.ml.feature import StringIndexer

# Index genre, tomatometer_status, and audience_status
genre_indexer = StringIndexer(inputCol="genre", outputCol="genre_indexed", handleInvalid="keep")
tomatometer_indexer = StringIndexer(inputCol="tomatometer_status", outputCol="tomatometer_status_indexed", handleInvalid="keep")
audience_indexer = StringIndexer(inputCol="audience_status", outputCol="audience_status_indexed", handleInvalid="keep")

# Apply indexers
df = genre_indexer.fit(df).transform(df)
df = tomatometer_indexer.fit(df).transform(df)
df = audience_indexer.fit(df).transform(df)

df.select("genre", "genre_indexed", "tomatometer_status", "tomatometer_status_indexed", "audience_status", "audience_status_indexed").show(5)


/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


+-------+-------------+------------------+--------------------------+---------------+-----------------------+
|  genre|genre_indexed|tomatometer_status|tomatometer_status_indexed|audience_status|audience_status_indexed|
+-------+-------------+------------------+--------------------------+---------------+-----------------------+
|Unknown|          0.0|                 1|                       0.0|              1|                    0.0|
|Unknown|          0.0|                 1|                       0.0|              1|                    0.0|
|Unknown|          0.0|                 1|                       0.0|              1|                    0.0|
|Unknown|          0.0|                 1|                       0.0|              1|                    0.0|
|Unknown|          0.0|                 1|                       0.0|              1|                    0.0|
+-------+-------------+------------------+--------------------------+---------------+-----------------------+
only showi

## **Step 3: Assemble Features**

All selected features (e.g., `numVotes`, `year`, `genre_indexed`, `tomatometer_rating`, etc.) are combined into a single `features` vector using **VectorAssembler**, which is the format Spark ML models expect.


In [5]:
from pyspark.ml.feature import VectorAssembler

# Define features
feature_cols = ["numVotes", "year", "tomatometer_rating", "audience_rating", 
                "genre_indexed", "tomatometer_status_indexed", "audience_status_indexed"]

# Assemble into a feature vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

df_ml = assembler.transform(df).select("features", "label")
df_ml.show(5)


+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1646.0,1935.0,78...|    1|
|[1646.0,1935.0,78...|    1|
|[1646.0,1935.0,78...|    1|
|[1646.0,1935.0,78...|    1|
|[1080.0,1935.0,78...|    1|
+--------------------+-----+
only showing top 5 rows



## **Step 4: Convert Target Column + : Split Data**

The target variable `label` is currently a boolean (True/False) and needs to be converted to integers (0 or 1) to work with **GBTClassifier**.

We split the dataset into:
- **80% training data** to train the model  
- **20% testing data** to evaluate the model performance.


In [6]:
from pyspark.sql.functions import col

# Convert label to Integer FIRST
df = df.withColumn("label", col("label").cast("integer"))

# Assemble features AFTER fixing label
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_ml = assembler.transform(df).select("features", "label")

# Now split data
train_data, test_data = df_ml.randomSplit([0.8, 0.2], seed=42)


## **Step 5: Train the Model**

We train a **Gradient Boosted Trees Classifier (GBTClassifier)** using Spark MLlib, which is effective for structured/tabular data.


In [ ]:
def objective(params):
    maxDepth = int(params['maxDepth'])  # tree depth
    maxIter = int(params['maxIter'])  # boosting iterations
    stepSize = params['stepSize']  # LR

    gbt = GBTClassifier(
        featuresCol="features",
        labelCol="label",
        maxDepth=maxDepth,
        maxIter=maxIter,
        stepSize=stepSize
    )

    model = gbt.fit(train_data)
    predictions = model.transform(test_data)
    evaluator = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")
    roc_auc = evaluator.evaluate(predictions)

    return {'loss': -roc_auc, 'status': STATUS_OK}

search_space = {
    'maxDepth': hp.quniform('maxDepth', 3, 10, 1),  
    'maxIter': hp.quniform('maxIter', 10, 100, 10),  
    'stepSize': hp.uniform('stepSize', 0.01, 0.2) 
}

trials = Trials()
best_params = fmin(
    fn=objective,       
    space=search_space,
    algo=tpe.suggest,   
    max_evals=20,        
    trials=trials
)

best_params['maxDepth'] = int(best_params['maxDepth'])
best_params['maxIter'] = int(best_params['maxIter'])

print("Best Hyperparameters Found:")
print(f"Best maxIter: {best_params['maxIter']}")
print(f"Best maxDepth: {best_params['maxDepth']}")
print(f"Best stepSize: {best_params['stepSize']}")

best_gbt = GBTClassifier(
    featuresCol="features",
    labelCol="label",
    maxDepth=best_params['maxDepth'],
    maxIter=best_params['maxIter'],
    stepSize=best_params['stepSize']
)

best_model = best_gbt.fit(train_data)
predictions = best_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predictions)
print(f"Best Model ROC-AUC Score: {roc_auc:.4f}")

evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_acc.evaluate(predictions)
print(f"Best Model Accuracy: {accuracy:.4f}")

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

  5%|▌         | 1/20 [00:28<09:03, 28.59s/trial, best loss: -0.7121905832286793]

 10%|█         | 2/20 [01:04<09:47, 32.65s/trial, best loss: -0.7227094791125354]

 20%|██        | 4/20 [01:49<06:33, 24.62s/trial, best loss: -0.7338314315202957]

 35%|███▌      | 7/20 [02:45<04:21, 20.12s/trial, best loss: -0.7338314315202957]

...

25/03/18 21:50:38 WARN DAGScheduler: Broadcasting large task binary with size 1000.5 KiB
25/03/18 21:50:39 WARN DAGScheduler: Broadcasting large task binary with size 1000.7 KiB
25/03/18 21:50:39 WARN DAGScheduler: Broadcasting large task binary with size 1006.3 KiB
25/03/18 21:50:39 WARN DAGScheduler: Broadcasting large task binary with size 1014.4 KiB
25/03/18 21:50:39 WARN DAGScheduler: Broadcasting large task binary with size 1023.3 KiB
25/03/18 21:50:39 WARN DAGScheduler: Broadcasting large task binary with size 1033.8 KiB
25/03/18 21:50:40 WARN DAGScheduler: Broadcasting large task binary with size 1025.5 KiB
25/03/18 21:50:40 WARN DAGScheduler: Broadcasting large task binary with size 1026.0 KiB
25/03/18 21:50:40 WARN DAGScheduler: Broadcasting large task binary with size 1026.5 KiB
25/03/18 21:50:40 WARN DAGScheduler: Broadcasting large task binary with size 1027.8 KiB
25/03/18 21:50:40 WARN DAGScheduler: Broadcasting large task binary with size 1029.8 KiB
25/03/18 21:50:40 WAR

Best Model ROC-AUC Score: 0.7496


25/03/18 21:51:31 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB


Best Model Accuracy: 0.6800


25/03/18 23:32:08 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 501577 ms exceeds timeout 120000 ms
25/03/18 23:32:08 WARN SparkContext: Killing executors is not supported by current scheduler.
25/03/18 23:32:14 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

## **Step 6: Make Predictions + evaluate performance**

We apply the trained model on the test set to generate predictions.

We evaluate the model using **ROC-AUC** and **Accuracy**:
- **ROC-AUC** gives an indication of how well the model distinguishes between highly rated and low-rated movies.
- **Accuracy** shows the percentage of correct classifications.




In [ ]:
#from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

#predictions = best_model.transform(test_data)  

#evaluator = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")
#roc_auc = evaluator.evaluate(predictions)
#print(f"Best Model ROC-AUC Score: {roc_auc:.4f}")

#evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
#accuracy = evaluator_acc.evaluate(predictions)
#print(f"Best Model Accuracy: {accuracy:.4f}")

+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|(7,[1,2,3],[1931....|    0|       1.0|
|(7,[1,2,3],[1933....|    1|       1.0|
|(7,[1,2,3],[1933....|    1|       1.0|
|(7,[1,2,3],[1933....|    1|       1.0|
|(7,[1,2,3],[1934....|    0|       1.0|
+--------------------+-----+----------+
only showing top 5 rows

ROC-AUC Score: 0.7146


In [ ]:
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
#accuracy = evaluator_acc.evaluate(predictions)
#print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.6554
